In [1]:
import pandas as pd
import numpy as np

### Step 1: Removing un-neccessary columns
#### These columns are to be kept from the original combined dataset:

- Schedule stats: Date,Comp,Round,Venue(Home/away),Result,GF,GA,Opponent,Xg,Xga,Possession, Formation

- Shooting stats: total_shots,SoT,SoT%,npXG (non-pen),G-Xg(Goals-XG),np:G-Xg,

- Goal Keeping stats: SoTA(Shots on target against), CS(clean sheets), PsXG, PSxG+/- 

- Passing stats: Cmp%,xAg,xA,KP,1/3(passes into final 3rd),PPA (passes into pen area),PrgP(progressive passes)

- Goal&Shot creation: SCA (shot creating actions), GCA (goal creating actions)

- Defensive: Tckls/Tckls won (Successfult Tckls %), Err (errors leading to shots)

- Possesions stats: Succ% (successful take on %), PrgDist, PrgC

- Misc: AerialsW% (aerial duel win %)

In [2]:
arsenal_df = pd.read_csv("../Arsenal_Stats/combined_22_26.csv")

cols_to_keep = ["Date", "Round", "Venue", "Result", "GF", "GA", "Opponent",
                "xG", "xGA", "Poss", "Formation", "('Standard', 'Sh')", "('Standard', 'SoT')",
                 "('Standard', 'SoT%')", "('Expected', 'npxG')", "('Expected', 'G-xG')",
                  "('Expected', 'np:G-xG')", "('Performance', 'SoTA')", "('Performance', 'CS')",
                    "('Performance', 'PSxG')", "('Performance', 'PSxG+/-')", "('Total', 'Cmp%')",
                "('Unnamed: 24_level_0', 'xAG')", "('Unnamed: 25_level_0', 'xA')", "('Unnamed: 26_level_0', 'KP')",
                "('Unnamed: 27_level_0', '1/3')", "('Unnamed: 28_level_0', 'PPA')", "('Unnamed: 30_level_0', 'PrgP')",
                "('SCA Types', 'SCA')", "('GCA Types', 'GCA')", "('Tackles', 'Tkl')","('Tackles', 'TklW')",
                "('Unnamed: 24_level_0', 'Err')", "('Take-Ons', 'Succ%')", "('Carries', 'PrgDist')", "('Carries', 'PrgC')", "('Aerial Duels', 'Won%')"]
                
new_arsenal_df = arsenal_df[cols_to_keep]
new_arsenal_df.head()

,Date,Round,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,...,"('Unnamed: 30_level_0', 'PrgP')","('SCA Types', 'SCA')","('GCA Types', 'GCA')","('Tackles', 'Tkl')","('Tackles', 'TklW')","('Unnamed: 24_level_0', 'Err')","('Take-Ons', 'Succ%')","('Carries', 'PrgDist')","('Carries', 'PrgC')","('Aerial Duels', 'Won%')"
0,2025-09-13,Matchweek 4,Home,W,3.0,0.0,Nott'ham Forest,1.8,0.2,54.0,...,50.0,28.0,6.0,13.0,11.0,0.0,45.8,1366.0,26.0,52.9
1,2025-08-31,Matchweek 3,Away,L,0.0,1.0,Liverpool,0.5,0.5,47.0,...,28.0,16.0,0.0,14.0,6.0,1.0,33.3,967.0,20.0,44.4
2,2025-08-23,Matchweek 2,Home,W,5.0,0.0,Leeds United,2.7,0.2,67.0,...,43.0,31.0,9.0,12.0,8.0,0.0,46.2,1260.0,30.0,75.0
3,2025-08-17,Matchweek 1,Away,W,1.0,0.0,Manchester Utd,1.3,1.5,39.0,...,24.0,14.0,2.0,18.0,10.0,0.0,32.1,619.0,12.0,44.4
4,2025-05-25,Matchweek 38,Away,W,2.0,1.0,Southampton,2.3,0.6,62.0,...,62.0,43.0,4.0,18.0,4.0,1.0,50.0,1069.0,22.0,51.9


In [3]:
new_arsenal_df.columns = ["Date", "Round", "Venue(Home/away)","Result","GF","GA","Opponent","Xg","Xga","Possession",
                        "Formation", "total_shots", "SoT","SoT%", "npxG", "G-xG","np:G-xG", "SoTA", 
                        "CS", "PSxG", "PSxG+/-", "Cmp%",
                        "xAG", "xA", "KP", "KP-1/3", "PPA", "PrgP",
                        "SCA", "GCA", "Tkl","TklW",
                        "Err", "Succ%", "PrgDist", "PrgC", "Aerial-Won%" ] # Renaming columns

# lets also add the team (we are trying to predict for) as the first column
new_arsenal_df.insert(2, "Team", "Arsenal") #replace with variable later

In [4]:

new_arsenal_df.head()


,Date,Round,Team,Venue(Home/away),Result,GF,GA,Opponent,Xg,Xga,...,PrgP,SCA,GCA,Tkl,TklW,Err,Succ%,PrgDist,PrgC,Aerial-Won%
0,2025-09-13,Matchweek 4,Arsenal,Home,W,3.0,0.0,Nott'ham Forest,1.8,0.2,...,50.0,28.0,6.0,13.0,11.0,0.0,45.8,1366.0,26.0,52.9
1,2025-08-31,Matchweek 3,Arsenal,Away,L,0.0,1.0,Liverpool,0.5,0.5,...,28.0,16.0,0.0,14.0,6.0,1.0,33.3,967.0,20.0,44.4
2,2025-08-23,Matchweek 2,Arsenal,Home,W,5.0,0.0,Leeds United,2.7,0.2,...,43.0,31.0,9.0,12.0,8.0,0.0,46.2,1260.0,30.0,75.0
3,2025-08-17,Matchweek 1,Arsenal,Away,W,1.0,0.0,Manchester Utd,1.3,1.5,...,24.0,14.0,2.0,18.0,10.0,0.0,32.1,619.0,12.0,44.4
4,2025-05-25,Matchweek 38,Arsenal,Away,W,2.0,1.0,Southampton,2.3,0.6,...,62.0,43.0,4.0,18.0,4.0,1.0,50.0,1069.0,22.0,51.9


### Adding Opponent form

- For each record we should add the opponents average number of points from their last 5 games at the time the game was being played. In order to investigate if opponent form is a decisive factor in the result.

In [16]:
# 1. go through every row in the df then get the "opponent"
# 2. Go to the prem_2018_2025.csv and retrieve the last 5 matches where "opponent" was either home or away
# 3. Find out if they were home or away and compare to FullTimeResult. If the opponent was home and FullTimeResult == H then ++1 for win
# OR ++3 for win ++1 for draw and 0 for loss (opponents avg no. pts from last 5 games)
combined_prem_df = pd.read_csv("../Prem_Stats/Prem_2018_2025.csv", parse_dates=['Date'], dayfirst=True) # This tells pandas to expect day/month/year

def get_points(row, team_col, result_col, home_col, away_col):
    """Given a row, return points for the team in that match."""
    if row[home_col] == team_col:
        if row[result_col] == 'H':
            return 3
        elif row[result_col] == 'D':
            return 1
        else:
            return 0
    elif row[away_col] == team_col:
        if row[result_col] == 'A':
            return 3
        elif row[result_col] == 'D':
            return 1
        else:
            return 0
    else:
        return None  # Should not happen
    
#team_df is the team we are looking at currently (this case Arsenal)
def calculate_opp_form(team_df, prem_df):
    # Convert to date_time objects so their dates can be compared
    team_df['Date'] = pd.to_datetime(arsenal_df['Date'])
    prem_df['Date'] = pd.to_datetime(prem_df['Date'], format=" %d/%m/%Y")

    opp_form_list = []

    for idx, row in team_df.iterrows():
        match_date = row['Date']
        opponent = row['Opponent']

        # Get last 5 matches for this opponent before this match date
        opp_matches = prem_df[
            ((prem_df['HomeTeam'] == opponent) | (prem_df['AwayTeam'] == opponent)) &
            (prem_df['Date'] < match_date) & 
            (prem_df['Date'] > (match_date - pd.Timedelta(days=365))) # Set a limit to only look back at matches from the past year of this match
        ].sort_values('Date', ascending=False).head(5) # 5 most recent matches of opponent

        # Setting this 1 year prior limit could lead to especially promoted teams not having enough data to give an accurate reprsentation of their form
        # This should be something to consider
        points = []
        for _, opp_row in opp_matches.iterrows():
            points.append(get_points(opp_row, opponent, "FullTimeResult", "HomeTeam", "AwayTeam"))
       
        avg_points = sum(points) / len(points) if points else None
        opp_form_list.append(avg_points)
    new_arsenal_df['Opponent_last5_AvgPts'] = opp_form_list
    return new_arsenal_df

arsenal_df_2 = calculate_opp_form(new_arsenal_df, combined_prem_df)
new_arsenal_df.head()

C:\Users\alexh\AppData\Local\Temp\ipykernel_11064\1375574467.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Date'] = pd.to_datetime(arsenal_df['Date'])
C:\Users\alexh\AppData\Local\Temp\ipykernel_11064\1375574467.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_arsenal_df['Opponent_last5_AvgPts'] = opp_form_list


,Date,Round,Team,Venue(Home/away),Result,GF,GA,Opponent,Xg,Xga,...,SCA,GCA,Tkl,TklW,Err,Succ%,PrgDist,PrgC,Aerial-Won%,Opponent_last5_AvgPts
0,2025-09-13,Matchweek 4,Arsenal,Home,W,3.0,0.0,Nott'ham Forest,1.8,0.2,...,28.0,6.0,13.0,11.0,0.0,45.8,1366.0,26.0,52.9,1.4
1,2025-08-31,Matchweek 3,Arsenal,Away,L,0.0,1.0,Liverpool,0.5,0.5,...,16.0,0.0,14.0,6.0,1.0,33.3,967.0,20.0,44.4,1.6
2,2025-08-23,Matchweek 2,Arsenal,Home,W,5.0,0.0,Leeds United,2.7,0.2,...,31.0,9.0,12.0,8.0,0.0,46.2,1260.0,30.0,75.0,3.0
3,2025-08-17,Matchweek 1,Arsenal,Away,W,1.0,0.0,Manchester Utd,1.3,1.5,...,14.0,2.0,18.0,10.0,0.0,32.1,619.0,12.0,44.4,0.8
4,2025-05-25,Matchweek 38,Arsenal,Away,W,2.0,1.0,Southampton,2.3,0.6,...,43.0,4.0,18.0,4.0,1.0,50.0,1069.0,22.0,51.9,0.4


### Finding Past 3 meeting results
- I think the average number of points the predicting team obtained in the last x meetings would be a suitable figure.

In [38]:
def past_matches(team_df, prem_df):
    # Convert to date_time objects so their dates can be compared
    team_df['Date'] = pd.to_datetime(arsenal_df['Date'])
    prem_df['Date'] = pd.to_datetime(prem_df['Date'], format=" %d/%m/%Y")
    predicting_team = team_df['Team'][0]
    meetings_list = []

    for _, row in team_df.iterrows():
        match_date = row['Date']
        opponent = row['Opponent']
        

        opp_matches = prem_df[
            ( ((prem_df['HomeTeam'] == opponent) & (prem_df['AwayTeam'] == predicting_team)) |
            ((prem_df['HomeTeam'] == predicting_team) & (prem_df['AwayTeam'] == opponent)) ) &
            (prem_df['Date'] < match_date)
        ].sort_values('Date', ascending=False).head(5) # 5 most recent matches of opponent

        if opponent == "Manchester Utd":
            print(opp_matches)
        points = []
        for _, opp_row in opp_matches.iterrows():
            points.append(get_points(opp_row, opponent, "FullTimeResult", "HomeTeam", "AwayTeam"))
        
        print("points: ", points)
        avg_points = sum(points) / len(points) if points else None
        meetings_list.append(avg_points)
        print("meeting list: ", meetings_list)
    
    
    new_arsenal_df["Team_avg_ppMeetings"] = meetings_list
    return new_arsenal_df

new_arsenal_df = past_matches(new_arsenal_df, combined_prem_df)
new_arsenal_df.head()
        


points:  [1, 0, 0, 0, 3]
meeting list:  [0.8]
points:  [1, 1, 0, 1, 1]
meeting list:  [0.8, 0.8]
points:  [0, 0, 0, 0, 0]
meeting list:  [0.8, 0.8, 0.0]
         Season  MatchWeek       Date   Time        HomeTeam        AwayTeam  \
2561  2024-2025         28 2025-03-09  16:30  Manchester Utd         Arsenal   
2314  2024-2025         14 2024-12-04  20:15         Arsenal  Manchester Utd   
2265  2023-2024         37 2024-05-12  16:30  Manchester Utd         Arsenal   
1938  2023-2024          4 2023-09-03  16:30         Arsenal  Manchester Utd   
1717  2022-2023         21 2023-01-22  16:30         Arsenal  Manchester Utd   

      FullTimeHomeTeamGoals  FullTimeAwayTeamGoals FullTimeResult  \
2561                      1                      1              D   
2314                      2                      0              H   
2265                      0                      1              A   
1938                      3                      1              H   
1717                 

C:\Users\alexh\AppData\Local\Temp\ipykernel_11064\1631608080.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df['Date'] = pd.to_datetime(arsenal_df['Date'])
C:\Users\alexh\AppData\Local\Temp\ipykernel_11064\1631608080.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_arsenal_df["Team_avg_ppMeetings"] = meetings_list


,Date,Round,Team,Venue(Home/away),Result,GF,GA,Opponent,Xg,Xga,...,GCA,Tkl,TklW,Err,Succ%,PrgDist,PrgC,Aerial-Won%,Opponent_last5_AvgPts,Team_avg_ppMeetings
0,2025-09-13,Matchweek 4,Arsenal,Home,W,3.0,0.0,Nott'ham Forest,1.8,0.2,...,6.0,13.0,11.0,0.0,45.8,1366.0,26.0,52.9,1.4,0.8
1,2025-08-31,Matchweek 3,Arsenal,Away,L,0.0,1.0,Liverpool,0.5,0.5,...,0.0,14.0,6.0,1.0,33.3,967.0,20.0,44.4,1.6,0.8
2,2025-08-23,Matchweek 2,Arsenal,Home,W,5.0,0.0,Leeds United,2.7,0.2,...,9.0,12.0,8.0,0.0,46.2,1260.0,30.0,75.0,3.0,0.0
3,2025-08-17,Matchweek 1,Arsenal,Away,W,1.0,0.0,Manchester Utd,1.3,1.5,...,2.0,18.0,10.0,0.0,32.1,619.0,12.0,44.4,0.8,0.2
4,2025-05-25,Matchweek 38,Arsenal,Away,W,2.0,1.0,Southampton,2.3,0.6,...,4.0,18.0,4.0,1.0,50.0,1069.0,22.0,51.9,0.4,1.0
